## Libraries

In [1]:
import pandas as pd
import numpy as np
import folium as fl
import wikipedia
import pyttsx3
import random
import requests
import credentials

## GPS Search

In [3]:
# Define GPS Coordinates of Barcelona
# lat_input = 41.390205
# log_input = 2.154007

# Random values for lat and long
lat_input = random.uniform(40, 45)
log_input = random.uniform(0, 5)

# Define the radius of the circle
radius = 0.5

# Load worldcities dataset zip 'https://simplemaps.com/data/world-cities'
worldcities = pd.read_csv('worldcities.csv')

print(worldcities.head(10))

          city   city_ascii      lat       lng      country iso2 iso3  \
0        Tokyo        Tokyo  35.6839  139.7744        Japan   JP  JPN   
1      Jakarta      Jakarta  -6.2146  106.8451    Indonesia   ID  IDN   
2        Delhi        Delhi  28.6667   77.2167        India   IN  IND   
3       Manila       Manila  14.6000  120.9833  Philippines   PH  PHL   
4    São Paulo    Sao Paulo -23.5504  -46.6339       Brazil   BR  BRA   
5        Seoul        Seoul  37.5600  126.9900  South Korea   KR  KOR   
6       Mumbai       Mumbai  19.0758   72.8775        India   IN  IND   
7     Shanghai     Shanghai  31.1667  121.4667        China   CN  CHN   
8  Mexico City  Mexico City  19.4333  -99.1333       Mexico   MX  MEX   
9    Guangzhou    Guangzhou  23.1288  113.2590        China   CN  CHN   

         admin_name  capital  population          id  
0             Tōkyō  primary  39105000.0  1392685764  
1           Jakarta  primary  35362000.0  1360771077  
2             Delhi    admin  3

In [4]:
# Calculate the distance between the input coordinates and the coordinates of the cities
worldcities['distance'] = np.sqrt((worldcities['lat'] - lat_input) ** 2 + (worldcities['lng'] - log_input) ** 2)

# Find the closest city
closest_cities = worldcities.sort_values(by='distance').head(10)
closest_city = worldcities.loc[worldcities['distance'].idxmin()]

# Print the closest city
print(closest_city)

city           Calvisson
city_ascii     Calvisson
lat               43.785
lng               4.1922
country           France
iso2                  FR
iso3                 FRA
admin_name     Occitanie
capital              NaN
population        5833.0
id            1250194430
distance        0.125156
Name: 39040, dtype: object


## Knowledge Search

In [5]:
try:
    page = wikipedia.page(closest_city['city'], ',', closest_city['country'])
    print(page.summary)
except:
    try:
        page = wikipedia.page(closest_city['city'])
        print(page.summary)
    except:
        print('No information found')

Calvisson (French pronunciation: ​[kalvisɔ̃]; Provençal: Cauviçon) is a commune in the Gard department in the Occitanie region in southern France.
It lies between Nîmes, Montpellier, the Cevennes and the Camargue and has a strong Protestant history.


## Map data

In [6]:
# City Map with folium
# https://python-visualization.github.io/folium/

# Create a map of the city
city_map = fl.Map(location=[closest_city['lat'], closest_city['lng']], zoom_start=10)

# Add a marker to the map
fl.Marker([closest_city['lat'], closest_city['lng']], popup=closest_city['city'], icon=fl.Icon(color='blue', icon='cross'),).add_to(city_map)
fl.Marker([lat_input, log_input], popup='You are here', icon=fl.Icon(color='red', icon='cloud'),).add_to(city_map)

# Display the map
city_map
#city_map.save('city_map.html')  # <- HTML file in directory

##  Synthesize Speech

In [7]:
# Create the story
story = 'The closest city to your location is ' + closest_city['city'] + ' in ' + closest_city[
    'country'] + '. ' + page.summary
# Cut story after 4th sentence if possible and convert to string
shortstory = str(story.split('.', 4)[0:4])
# Concat story back together and remove brackets
shortstory = shortstory.replace('[', '').replace(']', '').replace("'", '')

shortstory

'The closest city to your location is Calvisson in France,  Calvisson (French pronunciation: \\u200bkalvisɔ̃; Provençal: Cauviçon) is a commune in the Gard department in the Occitanie region in southern France, \\nIt lies between Nîmes, Montpellier, the Cevennes and the Camargue and has a strong Protestant history, '

In [8]:
# Synthesize story with pyttsx3
tts = pyttsx3.init()
# tts speed rate
tts.setProperty('rate', 80)
tts.say(shortstory)
tts.runAndWait()

# Sort city name by length
city_name = closest_city['city']
city_name = city_name.split()
city_name = sorted(city_name, key=len)
city_name = city_name[-1]

In [9]:
# Get surroundings by google maps api
api_key = credentials.google_api_key

lat = 41.382818
lng =  2.182477
radius = 1000               # -33.8670522,151.1957362&radius=500&
types = 'All'
search_name = ''

# Google Maps directions API endpoint
prompt = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&types={types}&name={search_name}&key={api_key}'
# Send request and get response
response = requests.get(prompt)
# Get response data as Python object
data = response.json()
results = data['results']
# Parse the results to a dataframe
df = pd.DataFrame.from_dict(results)
df

,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,reference,scope,types,vicinity,business_status,opening_hours,plus_code,rating,user_ratings_total,price_level
0,"{'location': {'lat': 41.3873974, 'lng': 2.1685...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Barcelona,"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJ5TCOcRaYpBIRCmZHTz37sEQ,ChIJ5TCOcRaYpBIRCmZHTz37sEQ,GOOGLE,"[locality, political]",Barcelona,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'location': {'lat': 41.3843884, 'lng': 2.1785...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,H10 Montcada,"[{'height': 528, 'html_attributions': ['<a hre...",ChIJjSlZP_mipBIR0_YxNCqjkTs,ChIJjSlZP_mipBIR0_YxNCqjkTs,GOOGLE,"[lodging, point_of_interest, establishment]","Via Laietana, 24, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95MH+QC Barcelona, Spain', ...",4.5,804.0,NaN
2,"{'location': {'lat': 41.3849706, 'lng': 2.1777...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Hotel Central,"[{'height': 787, 'html_attributions': ['<a hre...",ChIJg1-2b_mipBIRirLgoXI-GXU,ChIJg1-2b_mipBIRirLgoXI-GXU,GOOGLE,"[bar, spa, lodging, restaurant, point_of_inter...","Via Laietana, 30, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95MH+X4 Barcelona, Spain', ...",4.5,1116.0,NaN
3,"{'location': {'lat': 41.37972699999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Oriente Atiram Hotel Barcelona,"[{'height': 679, 'html_attributions': ['<a hre...",ChIJofXmHliipBIR4iyHDfDXnJA,ChIJofXmHliipBIR4iyHDfDXnJA,GOOGLE,"[lodging, point_of_interest, establishment]","Rambla dels Caputxins, 45, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95HF+VM Barcelona, Spain', ...",4.0,1604.0,NaN
4,"{'location': {'lat': 41.37991050000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel España,"[{'height': 1541, 'html_attributions': ['<a hr...",ChIJY6oYcliipBIRRvaQXvMe0KY,ChIJY6oYcliipBIRRvaQXvMe0KY,GOOGLE,"[lodging, point_of_interest, establishment]","Carrer de Sant Pau, 9-11, Barcelona",OPERATIONAL,NaN,"{'compound_code': '95HF+X4 Barcelona, Spain', ...",4.4,886.0,NaN
5,"{'location': {'lat': 41.3867134, 'lng': 2.1838...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,K+K Hotel Picasso El Born,"[{'height': 2252, 'html_attributions': ['<a hr...",ChIJc2TJ9P2ipBIRA65Mte-Mak8,ChIJc2TJ9P2ipBIRA65Mte-Mak8,GOOGLE,"[lodging, point_of_interest, establishment]","Passeig de Picasso, 26, 30, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95PM+MG Barcelona, Spain', ...",4.4,1029.0,NaN
6,"{'location': {'lat': 41.38095999999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,RAMBLAS HOTEL powered by Vincci Hoteles,"[{'height': 970, 'html_attributions': ['<a hre...",ChIJbSlW1PeipBIRe0EQZnJJWUc,ChIJbSlW1PeipBIRe0EQZnJJWUc,GOOGLE,"[lodging, point_of_interest, establishment]","Carrer de l'Hospital, 26, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95JC+9R Barcelona, Spain', ...",4.3,546.0,NaN
7,"{'location': {'lat': 41.3878667, 'lng': 2.1733...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel HCC Montblanc,"[{'height': 1780, 'html_attributions': ['<a hr...",ChIJHagl9fCipBIRdsXraeNfmvw,ChIJHagl9fCipBIRdsXraeNfmvw,GOOGLE,"[lodging, point_of_interest, establishment]","Via Laietana, 61, Barcelona",OPERATIONAL,{'open_now': True},"{'compound_code': '95QF+48 Barcelona, Spain', ...",4.3,2145.0,NaN
8,"{'location': {'lat': 41.3825393, 'lng': 2.1789...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.g

In [10]:
df = df[['geometry', 'name', 'place_id', 'types', 'vicinity', 'business_status', 'rating', 'user_ratings_total', 'opening_hours']]

# Extract lat
df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'])
df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'])
df = df.drop(columns=['geometry'])

# Extract open_now
# Display the dataframe
df

/var/folders/sl/c5pyww8j61j6_9_z4twp2ygc0000gn/T/ipykernel_32121/2554590475.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'])
/var/folders/sl/c5pyww8j61j6_9_z4twp2ygc0000gn/T/ipykernel_32121/2554590475.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'])


,name,place_id,types,vicinity,business_status,rating,user_ratings_total,opening_hours,lat,lng
0,Barcelona,ChIJ5TCOcRaYpBIRCmZHTz37sEQ,"[locality, political]",Barcelona,NaN,NaN,NaN,NaN,41.387397,2.168568
1,H10 Montcada,ChIJjSlZP_mipBIR0_YxNCqjkTs,"[lodging, point_of_interest, establishment]","Via Laietana, 24, Barcelona",OPERATIONAL,4.5,804.0,{'open_now': True},41.384388,2.178517
2,Grand Hotel Central,ChIJg1-2b_mipBIRirLgoXI-GXU,"[bar, spa, lodging, restaurant, point_of_inter...","Via Laietana, 30, Barcelona",OPERATIONAL,4.5,1116.0,{'open_now': True},41.384971,2.177765
3,Oriente Atiram Hotel Barcelona,ChIJofXmHliipBIR4iyHDfDXnJA,"[lodging, point_of_interest, establishment]","Rambla dels Caputxins, 45, Barcelona",OPERATIONAL,4.0,1604.0,{'open_now': True},41.379727,2.174221
4,Hotel España,ChIJY6oYcliipBIRRvaQXvMe0KY,"[lodging, point_of_interest, establishment]","Carrer de Sant Pau, 9-11, Barcelona",OPERATIONAL,4.4,886.0,NaN,41.379911,2.172841
5,K+K Hotel Picasso El Born,ChIJc2TJ9P2ipBIRA65Mte-Mak8,"[lodging, point_of_interest, establishment]","Passeig de Picasso, 26, 30, Barcelona",OPERATIONAL,4.4,1029.0,{'open_now': True},41.386713,2.183873
6,RAMBLAS HOTEL powered by Vincci Hoteles,ChIJbSlW1PeipBIRe0EQZnJJWUc,"[lodging, point_of_interest, establishment]","Carrer de l'Hospital, 26, Barcelona",OPERATIONAL,4.3,546.0,{'open_now': True},41.380960,2.172125
7,Hotel HCC Montblanc,ChIJHagl9fCipBIRdsXraeNfmvw,"[lodging, point_of_interest, establishment]","Via Laietana, 61, Barcelona",OPERATIONAL,4.3,2145.0,{'open_now': True},41.387867,2.173367
8,Mercer Hotel Barcelona,ChIJQ8ly2viipBIRZANMAFMePb8,"[lodging, point_of_interest, establishment]","Carrer dels Lledó, 7, Barcelona",OPERATIONAL,4.6,313.0,NaN,41.382539,2.178967
9,Hotel Neri Relais & Chateaux‎,ChIJWU9gM_iipBIRBo0JgzCmYjI,"[lodging, point_of_interest, establishment]","Carrer de Sant Sever, 5, Barcelona",OPERATIONAL,4.4,471.0,{'open_now': True},41.383172,2.175381


In [11]:
# Visualize first 10 results from the csv table on a map
# df = df[:10]
# Generate location between items in the dataframe - find middle point
lat = df['lat'].mean()
lng = df['lng'].mean()

# Create a map of the city
city_map = fl.Map(location=[lat, lng], zoom_start=15)

# Add a marker for each item in the dataframe
for i in range(0, len(df)):
    fl.Marker([df.iloc[i]['lat'], df.iloc[i]['lng']], popup=df.iloc[i]['name'], icon=fl.Icon(color='blue', icon='cross' ),).add_to(city_map)

# Display the map
city_map


In [12]:
import pyaudio
import sys
import wave

Could not import the PyAudio C module 'pyaudio._portaudio'.


ImportError: dlopen(/Users/nilsjennissen/PycharmProjects/location_storyteller/venv/lib/python3.9/site-packages/pyaudio/_portaudio.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '_PaMacCore_SetupChannelMap'

In [13]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

audio = pyaudio.PyAudio()

# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True, frames_per_buffer=CHUNK)

print("recording...")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

# create wave file
wf = wave.open(f'data/{WAVE_OUTPUT_FILENAME}', 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

stream.stop_stream()
print("finished recording")

NameError: name 'pyaudio' is not defined

In [20]:
sound = wave.open("data/output.wav")
p = pyaudio.PyAudio()
print(f"Start playing {sound.getnchannels()} channels at {sound.getframerate()} Hz")
chunk = 1024
stream = p.open(format=p.get_format_from_width(sound.getsampwidth()),
                channels=sound.getnchannels(),
                rate=sound.getframerate(),
                output=True)
data = sound.readframes(chunk)
while True:
    if data != '':
        stream.write(data)
        data = sound.readframes(chunk)

    if data == b'':
        break

stream.stop_stream()
stream.close()
p.terminate()

print("Finished playing")

EOFError: 

In [ ]:
# Function to get surrounding maps elements
api_key = credentials.google_api_key

lat = 41.382818
lng =  2.182477
radius = 1000               # -33.8670522,151.1957362&radius=500&
types = 'All'
search_name = ''

def get_map_items(lat, lng, radius, types, search_name):
    # Google Maps directions API endpoint
    prompt = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&types={types}&name={search_name}&key={api_key}'
    # Send request and get response
    response = requests.get(prompt)
    # Get response data as Python object
    data = response.json()
    results = data['results']
    # Parse the results to a dataframe
    df = pd.DataFrame.from_dict(results)
    # Prepare df with selected columns
    df = df[['geometry', 'name', 'place_id', 'types', 'vicinity', 'business_status', 'rating', 'user_ratings_total', 'opening_hours']]
    # Extract lat
    df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'])
    df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'])
    df = df.drop(columns=['geometry'])
    # Display the dataframe
    return df


In [ ]:
# Get surrounding maps elements
test = get_map_items(lat, lng, radius, types, search_name)
test

In [12]:
# Function to get surrounding maps elements
api_key = credentials.google_api_key

lat = 41.382818
lng =  2.182477
radius = 1000               # -33.8670522,151.1957362&radius=500&
types = 'All'
search_name = ''

def get_map_items(lat, lng, radius, types, search_name):
    # Google Maps directions API endpoint
    prompt = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&types={types}&name={search_name}&key={api_key}'
    # Send request and get response
    response = requests.get(prompt)
    # Get response data as Python object
    data = response.json()
    results = data['results']
    # Parse the results to a dataframe
    df = pd.DataFrame.from_dict(results)
    # Prepare df with selected columns
    df = df[['geometry', 'name', 'place_id', 'types', 'vicinity', 'business_status', 'rating', 'user_ratings_total', 'opening_hours']]
    # Extract lat
    df['lat'] = df['geometry'].apply(lambda x: x['location']['lat'])
    df['lng'] = df['geometry'].apply(lambda x: x['location']['lng'])
    df = df.drop(columns=['geometry'])
    # Display the dataframe
    return df


In [13]:
# Get surrounding maps elements
test = get_map_items(lat, lng, radius, types, search_name)
test

,name,place_id,types,vicinity,business_status,rating,user_ratings_total,opening_hours,lat,lng
0,Barcelona,ChIJ5TCOcRaYpBIRCmZHTz37sEQ,"[locality, political]",Barcelona,NaN,NaN,NaN,NaN,41.387397,2.168568
1,H10 Montcada,ChIJjSlZP_mipBIR0_YxNCqjkTs,"[lodging, point_of_interest, establishment]","Via Laietana, 24, Barcelona",OPERATIONAL,4.5,790.0,{'open_now': True},41.384388,2.178517
2,Grand Hotel Central,ChIJg1-2b_mipBIRirLgoXI-GXU,"[bar, lodging, spa, restaurant, food, point_of...","Via Laietana, 30, Barcelona",OPERATIONAL,4.5,1107.0,{'open_now': True},41.384971,2.177765
3,Oriente Atiram Hotel Barcelona,ChIJofXmHliipBIR4iyHDfDXnJA,"[lodging, point_of_interest, establishment]","Rambla dels Caputxins, 45, Barcelona",OPERATIONAL,4.0,1590.0,{'open_now': True},41.379727,2.174221
4,Hotel España,ChIJY6oYcliipBIRRvaQXvMe0KY,"[lodging, point_of_interest, establishment]","Carrer de Sant Pau, 9-11, Barcelona",OPERATIONAL,4.4,879.0,NaN,41.379911,2.172841
5,K+K Hotel Picasso El Born,ChIJc2TJ9P2ipBIRA65Mte-Mak8,"[lodging, point_of_interest, establishment]","Passeig de Picasso, 26, 30, Barcelona",OPERATIONAL,4.4,1014.0,{'open_now': True},41.386713,2.183873
6,RAMBLAS HOTEL powered by Vincci Hoteles,ChIJbSlW1PeipBIRe0EQZnJJWUc,"[lodging, point_of_interest, establishment]","Carrer de l'Hospital, 26, Barcelona",OPERATIONAL,4.3,540.0,{'open_now': True},41.380960,2.172125
7,Hotel HCC Montblanc,ChIJHagl9fCipBIRdsXraeNfmvw,"[lodging, point_of_interest, establishment]","Via Laietana, 61, Barcelona",OPERATIONAL,4.3,2112.0,{'open_now': True},41.387867,2.173367
8,Mercer Hotel Barcelona,ChIJQ8ly2viipBIRZANMAFMePb8,"[lodging, point_of_interest, establishment]","Carrer dels Lledó, 7, Barcelona",OPERATIONAL,4.6,311.0,NaN,41.382539,2.178967
9,Hotel Neri Relais & Chateaux‎,ChIJWU9gM_iipBIRBo0JgzCmYjI,"[lodging, point_of_interest, establishment]","Carrer de Sant Sever, 5, Barcelona",OPERATIONAL,4.4,467.0,{'open_now': True},41.383172,2.175381
